In [1]:
'''
Lucas Atayde, Alp Burke

File responsible for cleaning up our data pipeline. This will prepare our API to query the
data.
'''

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [22]:
usaa = pd.read_csv("./raw/US_Accidents_Dec19.csv", skiprows=range( int(2974335 * .01), 2974335))
# usaa = pd.read_csv("data_prep/raw/US_Accidents_Dec19.csv")


In [3]:
# Step 1: Convert accident ids to integers
usaa["ID"] = usaa["ID"].apply(lambda id: int(id[2:]))

# Drop unnecessary data. Makes computation slower and adds noise
# Country is the same for ALL.
# Source might effect data quality but ehhhh
usaa = usaa.drop(columns=["Source", "Country"])


In [4]:
# Fill in data with normal distribution
# TODO: Make this more sophisticated using nearest neighbors?
t_mean = usaa["Temperature(F)"].mean()
t_std = usaa["Temperature(F)"].std()
usaa["Temperature(F)"] = usaa["Temperature(F)"].apply(lambda val: np.random.normal(t_mean, t_std) if np.isnan(val) else val)

In [5]:
def make_categorical(frame, col):
    frame[col] = frame[col].astype("category").cat.codes

def num_missing(series):
    return series.isnull().sum()


In [6]:
'''
TMC codes about traffic? Not exactly sure, will convert to categorical data
'''
make_categorical(usaa, "TMC")


In [7]:
# End lat and End long are both pretty much empty / worthless
usaa = usaa.drop(columns="End_Lng")
usaa = usaa.drop(columns="End_Lat")


In [8]:
cat = "City"
print(num_missing(usaa[cat]))
#usaa[cat].hist()
usaa[cat].unique()

0


array(['Dayton', 'Reynoldsburg', 'Williamsburg', 'Westerville',
       'Columbus', 'Miamisburg', 'Johnstown', 'Dublin',
       'Canal Winchester', 'Lebanon', 'Sabina', 'Goshen', 'Waynesville',
       'Springboro', 'Carroll', 'Oregonia', 'Independence',
       'Maple Heights', 'Grove City', 'Findlay', 'Fairborn', 'Piqua',
       'New Richmond', 'Batavia', 'Galena', 'Barberton', 'Akron',
       'Mansfield', 'Cleveland', 'Bethel', 'Pataskala', 'Seville',
       'Bellville', 'Troy', 'Wilmington', 'North Olmsted', 'Groveport',
       'Orient', 'Tiro', 'Strongsville', 'Beavercreek', 'Circleville',
       'Peebles', 'Hurricane', 'Xenia', 'Elyria', 'Pickerington',
       'Burbank', 'Zanesville', 'North Baltimore', 'Mount Sterling',
       'Tipp City', 'Moraine', 'Hilliard', 'Bowling Green', 'Maineville',
       'Vermilion', 'Milford', 'Brookpark', 'Fraziers Bottom',
       'Cambridge', 'Delaware', 'Moscow', 'Oberlin', 'Sardinia',
       'Lewis Center', 'Urbancrest', 'Brecksville', 'Frankfort',

In [24]:
usaa = usaa.dropna(subset=["City", "Zipcode"])

# We only want the first five digits of zipcode.
usaa["Zipcode"].apply(lambda zip: zip[:5])


0             45424
1        43068-3402
2             45176
3             45417
4             45459
            ...    
29738         94578
29739         94580
29740         95012
29741    95020-9510
29742         92346
Name: Zipcode, Length: 29738, dtype: object